In [1]:
import pandas as pd

from statistics import mean
import statistics
import numpy as np
import os
import subprocess
import re
import json
from tabulate import tabulate

In [2]:

with open("all_ast_results.json", "r") as fd:
    results = json.load(fd)

#################

func_results = {}


for data in results:
    opt = data["opt"]
    filename = data["filename"]
    for func, result in data["results"].items():
        func_name = f'{filename}:{func}:O{opt}'
        if func_name not in func_results:
            func_results[func_name] = {}
        for tool in ["w2c2", "wasm2c", "ghidra", 'retdec']:
            try:
                func_results[func_name][tool] = { "opt": int(opt),
                                 "nqc": result[tool]["nqc"], 
                                 "ted": result[tool]["ted"]}
            except:
                print(opt, tool, func_name, result)
                print()
            #if result[tool]["ted"]< 0 and opt != 2:
            #    print(opt, tool, func_name)

func_names = []
rows = []
for func_name, columns in func_results.items():
    func_names.append(func_name)
    rows.append(columns)
        
d = pd.json_normalize(rows)
df = pd.DataFrame(d)
df.columns = df.columns.str.split('.', expand=True)
# print(df.loc[0])
df["func"] = func_names
df = df.set_index("func")

#pd.set_option('display.max_rows', None)
#display(df['ghidra'])
#pd.reset_option('display.max_rows')



#####



for metrix in ["nqc", "ted"]:
    table = [
        ["metrix", "Tool", "Opt", "Total func", "AVG", "STD"]
    ]
    for tool in ["w2c2", "wasm2c", "ghidra", 'retdec']:
        for opt in [0,1,2]:
            opt_tool_df = df[tool][(df[tool]["opt"] == opt)]
            cnt = round(opt_tool_df[(opt_tool_df[metrix] > 0)][metrix].count(), 3)
            avg = round(opt_tool_df[(opt_tool_df[metrix] > 0)][metrix].mean(), 3)
            std = round(opt_tool_df[(opt_tool_df[metrix] > 0)][metrix].std(), 3)
            table.append(
                [metrix, tool, opt, cnt, avg, std]
            )
    print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))        

╒══════════╤════════╤═══════╤══════════════╤═══════╤═══════╕
│ metrix   │ Tool   │   Opt │   Total func │   AVG │   STD │
╞══════════╪════════╪═══════╪══════════════╪═══════╪═══════╡
│ nqc      │ w2c2   │     0 │          145 │ 0.101 │ 0.074 │
├──────────┼────────┼───────┼──────────────┼───────┼───────┤
│ nqc      │ w2c2   │     1 │          145 │ 0.101 │ 0.077 │
├──────────┼────────┼───────┼──────────────┼───────┼───────┤
│ nqc      │ w2c2   │     2 │          145 │ 0.099 │ 0.078 │
├──────────┼────────┼───────┼──────────────┼───────┼───────┤
│ nqc      │ wasm2c │     0 │          145 │ 0.049 │ 0.023 │
├──────────┼────────┼───────┼──────────────┼───────┼───────┤
│ nqc      │ wasm2c │     1 │          145 │ 0.202 │ 0.141 │
├──────────┼────────┼───────┼──────────────┼───────┼───────┤
│ nqc      │ wasm2c │     2 │          145 │ 0.198 │ 0.148 │
├──────────┼────────┼───────┼──────────────┼───────┼───────┤
│ nqc      │ ghidra │     0 │          145 │ 0.573 │ 0.26  │
├──────────┼────────┼───

In [6]:

with open("all_readability_results.json", "r") as fd:
    rresults = json.load(fd)

    
####

func_rresults = {}


for data in rresults:
    opt = data["opt"]
    filename = data["filename"]
    for func, result in data["results"].items():
        func_name = f'{filename}:{func}:O{opt}'
        if func_name not in func_rresults:
            func_rresults[func_name] = {}
        for tool in ["w2c2", "wasm2c", "ghidra", 'retdec']:
            try:
                func_rresults[func_name][tool] = result[tool]
                func_rresults[func_name][tool]["opt"] = int(opt)
            except:
                print(opt, tool, func_name, result)
                print()
            #if result[tool]["ted"]< 0 and opt != 2:
            #    print(opt, tool, func_name)

func_names = []
rows = []
for func_name, columns in func_rresults.items():
    func_names.append(func_name)
    rows.append(columns)
        
d = pd.json_normalize(rows)
df = pd.DataFrame(d)
df.columns = df.columns.str.split('.', expand=True)
# print(df.loc[0])
df["func"] = func_names
df = df.set_index("func")

#pd.set_option('display.max_rows', None)
#display(df['ghidra'])
#pd.reset_option('display.max_rows')

#####



for metrix in ["Lines of code",
               "# goto",
               "# cast",
               "# var",
               "# dead code"
              ]:
    table = [
        ["Tool", "Opt", 
         #"AVG", 
         "SUM", 
         #"STD"
        ]
    ]
    print(metrix)
    for tool in ["w2c2", "wasm2c", "ghidra", 'retdec']:
        for opt in [0,1,2]:
            opt_tool_df = df[tool][(df[tool]["opt"] == opt)]
            avg = round(opt_tool_df[metrix].mean(), 3)
            ss = round(opt_tool_df[metrix].sum(), 3)
            std = round(opt_tool_df[metrix].std(), 3)
            table.append(
                [tool, opt, 
                 #avg, 
                 ss, 
                 #std
                ]
            )
    
    print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))     
    


for metrix in [
               "Maximum nesting depth",
               "Halstead complexity difficulty measure",
               "McCabe cyclomatic complexity"
              ]:
    table = [
        ["Tool", "Opt", 
         "AVG", 
         #"SUM", 
         "STD"]
    ]
    print(metrix)
    for tool in ["w2c2", "wasm2c", "ghidra", 'retdec']:
        for opt in [0,1,2]:
            opt_tool_df = df[tool][(df[tool]["opt"] == opt)]
            avg = round(opt_tool_df[metrix].mean(), 3)
            ss = round(opt_tool_df[metrix].sum(), 3)
            std = round(opt_tool_df[metrix].std(), 3)
            table.append(
                [tool, opt, 
                 avg, 
                 #ss, 
                 std
                ]
            )
    
    print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))       


Lines of code
╒════════╤═══════╤═══════╕
│ Tool   │   Opt │   SUM │
╞════════╪═══════╪═══════╡
│ w2c2   │     0 │ 74473 │
├────────┼───────┼───────┤
│ w2c2   │     1 │ 24828 │
├────────┼───────┼───────┤
│ w2c2   │     2 │ 27440 │
├────────┼───────┼───────┤
│ wasm2c │     0 │ 72362 │
├────────┼───────┼───────┤
│ wasm2c │     1 │ 28332 │
├────────┼───────┼───────┤
│ wasm2c │     2 │ 30461 │
├────────┼───────┼───────┤
│ ghidra │     0 │  4818 │
├────────┼───────┼───────┤
│ ghidra │     1 │  7487 │
├────────┼───────┼───────┤
│ ghidra │     2 │  9249 │
├────────┼───────┼───────┤
│ retdec │     0 │ 10376 │
├────────┼───────┼───────┤
│ retdec │     1 │ 11290 │
├────────┼───────┼───────┤
│ retdec │     2 │ 15575 │
╘════════╧═══════╧═══════╛
# goto
╒════════╤═══════╤═══════╕
│ Tool   │   Opt │   SUM │
╞════════╪═══════╪═══════╡
│ w2c2   │     0 │   767 │
├────────┼───────┼───────┤
│ w2c2   │     1 │   738 │
├────────┼───────┼───────┤
│ w2c2   │     2 │   584 │
├────────┼───────┼───────┤
│ wasm2

In [31]:
with open("readability_entries.json", "r") as fd:
    entries = json.load(fd)

with open("metrics/results/new/em_output_src.json", "r") as fd:
    rresults = json.load(fd)

    
####

func_rresults = {}


for entry in entries:
    if entry["opt"] != 0:
        continue
    filename = entry["filename"]
    if filename.startswith("PL_"):
        filename = filename.replace("PL_", "")
    functions = entry["functions"]
    for func in functions:
        try:
            func_name = f'{filename}:{func}'
            if func_name not in func_rresults:
                func_rresults[func_name] = {}
            for key, data in rresults[filename+".c"].items():
                func_rresults[func_name][key] = data[func]
        except:
            print(filename, func, key)


func_names = []
rows = []
for func_name, columns in func_rresults.items():
    func_names.append(func_name)
    rows.append(columns)
        
d = pd.json_normalize(rows)
df = pd.DataFrame(d)
df.columns = df.columns.str.split('.', expand=True)
# print(df.loc[0])
df["func"] = func_names
df = df.set_index("func")

#pd.set_option('display.max_rows', None)
#display(df)
#pd.reset_option('display.max_rows')

#####


tool = "orig"

for metrix in ["Lines of code",
               "# goto",
               "# cast",
               "# var",
               "# dead code"
              ]:
    table = [
        ["Tool", 
         #"AVG", 
         "SUM", 
         #"STD"
        ]
    ]
    print(metrix)
    avg = round(df[metrix].mean(), 3)
    ss = round(df[metrix].sum(), 3)
    std = round(df[metrix].std(), 3)
    table.append(
        [tool, 
         #avg, 
         ss, 
         #std
        ]
    )

    print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))     
    


for metrix in [
               "Maximum nesting depth",
               "Halstead complexity difficulty measure",
               "McCabe cyclomatic complexity"
              ]:
    table = [
        ["Tool", 
         "AVG", 
         #"SUM", 
         "STD"]
    ]
    print(metrix)
    avg = round(df[metrix].mean(), 3)
    ss = round(df[metrix].sum(), 3)
    std = round(df[metrix].std(), 3)
    table.append(
        [tool, 
         avg, 
         #ss, 
         std
        ]
    )
    
    print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

Lines of code
╒════════╤═══════╕
│ Tool   │   SUM │
╞════════╪═══════╡
│ orig   │  2619 │
╘════════╧═══════╛
# goto
╒════════╤═══════╕
│ Tool   │   SUM │
╞════════╪═══════╡
│ orig   │     0 │
╘════════╧═══════╛
# cast
╒════════╤═══════╕
│ Tool   │   SUM │
╞════════╪═══════╡
│ orig   │    39 │
╘════════╧═══════╛
# var
╒════════╤═══════╕
│ Tool   │   SUM │
╞════════╪═══════╡
│ orig   │   194 │
╘════════╧═══════╛
# dead code
╒════════╤═══════╕
│ Tool   │   SUM │
╞════════╪═══════╡
│ orig   │     4 │
╘════════╧═══════╛
Maximum nesting depth
╒════════╤═══════╤═══════╕
│ Tool   │   AVG │   STD │
╞════════╪═══════╪═══════╡
│ orig   │ 0.952 │ 1.163 │
╘════════╧═══════╧═══════╛
Halstead complexity difficulty measure
╒════════╤════════╤════════╕
│ Tool   │    AVG │    STD │
╞════════╪════════╪════════╡
│ orig   │ 14.329 │ 18.067 │
╘════════╧════════╧════════╛
McCabe cyclomatic complexity
╒════════╤═══════╤═══════╕
│ Tool   │   AVG │   STD │
╞════════╪═══════╪═══════╡
│ orig   │ 3.966 │ 4.144 │
╘

In [ ]:
# df2 is for common functions between w2c2, wasm2c, retdec

In [ ]:
func_results = {}


for data in results:
    opt = data["opt"]
    filename = data["filename"]
    for func, result in data["results"].items():
        func_name = f'{filename}:{func}:O{opt}'
        if func_name not in func_results:
            func_results[func_name] = {}
        for tool in ["w2c2", "wasm2c", 'retdec']:
            try:
                func_results[func_name][tool] = { "opt": int(opt),
                                 "nqc": result[tool]["nqc"], 
                                 "ted": result[tool]["ted"]}
            except:
                print(opt, tool, func_name, result)
                print()
            #if result[tool]["ted"]< 0 and opt != 2:
            #    print(opt, tool, func_name)

func_names = []
rows = []
for func_name, columns in func_results.items():
    func_names.append(func_name)
    rows.append(columns)
        
d = pd.json_normalize(rows)
df = pd.DataFrame(d)
df.columns = df.columns.str.split('.', expand=True)
# print(df.loc[0])
df["func"] = func_names
df = df.set_index("func")

pd.set_option('display.max_rows', None)
display(df['r2-ghidra'])
pd.reset_option('display.max_rows')